In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive


In [0]:
text = open('at.txt', 'r').read()
text = text.lower()
train_text = text[:int(len(text)*0.8)]
test_text = text[int(len(text)*0.8)+1:]
# sent_tokenize_list = sent_tokenize(text)

In [3]:
#I have used Keras official github example of LSTM as reference for this part of assignment.
#https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io


Using TensorFlow backend.


In [4]:
#Creating indexes corresponding to each character for mathematical representation.
chars = sorted(list(set(train_text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(train_text) - maxlen, step):
    sentences.append(train_text[i: i + maxlen])
    next_chars.append(train_text[i + maxlen])
print('nuber of sequences:', len(sentences))

print('Vectorization...')
#vector representation of input and target for training, One hot representation for each sentence and target character.  
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


total chars: 64
nuber of sequences: 238988
Vectorization...


In [5]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop()
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...







In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)


In [7]:

model.fit(x, y,
          batch_size=128,
          epochs=10,
          callbacks=[print_callback])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/10
238988/238988 [==============================] - 194s 811us/step - loss: 2.3657

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "is leadership, this situation will not g"
is leadership, this situation will not got the going to the got the got the going the going to the and the the the going to the going to me the got that the got the hat and i got the best the going to the going the going to what the going to the bes and the going to whe hand the going to me the going to the going to the going to the going to the and and the sout the going to the got the be the going to the hat ing to the going to the go
----- diversity: 0.5
----- Generating with seed: "is leadership, this situation will not g"
is leadership, this situation will not got the se pellith the made the eread for the bill as somant of so thing to know the kes and the wall it the cin

In [0]:
# As perplexity is exponential of entropy. I take exponential of cross entropy loss to the power -1/n..
# Perplexity : 113 (Done by calculator). 